In [24]:
%reload_ext autoreload
%autoreload 2

In [26]:
from pathlib import Path
import cv2
import matplotlib.pyplot as plt
from classic_segmentation import ClassicSegmentation, ImageDivider, open_cv_image, save_cv_image
from utils import save_image, random_label_probability
import random

In [27]:
root_folder = Path("Dataset")
ext = '.tif'  # source file extention
ext_save = '.png'  # save file extention
root_folder_save_class = Path('dataset_classification')
folder_save_segm = Path('dataset_segmentation')
segm_images = folder_save_segm / "images"; segm_images.mkdir(parents=True, exist_ok=True)
segm_labels = folder_save_segm / "labels"; segm_labels.mkdir(parents=True, exist_ok=True)

a dict with custom settings for each fruit you can put custom paramenter for the segmentation phase (segm) and for image division (divider)

In [28]:
settings = {"Chili Pepper":{
                "divider": {"min_area": 0.1},
            },
            "Apple": {
                "segm": {"floodfill": False},
            },
            "default": {
                "divider": {"min_area": .3},
            },
           }

In [29]:
def classic_segmenter(img_path, **setting):
    img = open_cv_image(img_path)
    mask = ClassicSegmentation(img, **setting.get('segm', {})).get_mask()
    imgs = ImageDivider(img, mask, **setting.get('divider', {})).images()
    return imgs

In [30]:
def process_folder_class(folder, **settings):
    n_images = 0
    folder_save = root_folder_save_class/folder.name
    folder_save.mkdir(parents=True, exist_ok=True)
    setting = settings.get(folder.name, settings.get("default", {}))
    for img_path in folder.glob(f"*{ext}"):
        for i, fruit_img in enumerate(classic_segmenter(img_path, **setting)):
            n_images += 1
            path = folder_save/f"{img_path.name[:-4]}_{i}{ext_save}"
            save_cv_image(str(path), fruit_img)
    return n_images

In [31]:
def process_folder_segm(folder, **settings):
    setting = settings.get(folder.name, settings.get("default", {}))
    for img_path in folder.glob(f"*{ext}"):
        mask = ClassicSegmentation(open_cv_image(img_path), **setting.get('segm', {})).get_mask()
        mask_path = segm_labels/f"{img_path.name[:-4]}.{ext_save}"
        save_cv_image(str(mask_path), mask)
        # copy the original image in the dataset after converting format
        new_img_path = str(segm_images/f"{img_path.name[:-4]}{ext_save}")
        save_cv_image(new_img_path, cv2.imread(str(img_path)))
    return n_images

In [32]:
for f in root_folder.iterdir():
    n_images = process_folder_class(f, **settings)
    print("Done Classfication processing for", f, "saved: ", n_images)
    process_folder_segm(f, **settings)
    print("Done segmentation processing for", f)

Done Classfication processing for ../Dataset/Zucchini saved:  100
Done segmentation processing for ../Dataset/Zucchini
Done Classfication processing for ../Dataset/Kiwifruit saved:  100
Done segmentation processing for ../Dataset/Kiwifruit
Done Classfication processing for ../Dataset/Lime saved:  106
Done segmentation processing for ../Dataset/Lime
Done Classfication processing for ../Dataset/Apple saved:  102
Done segmentation processing for ../Dataset/Apple
Done Classfication processing for ../Dataset/Cucumber saved:  100
Done segmentation processing for ../Dataset/Cucumber
Done Classfication processing for ../Dataset/Plum saved:  146
Done segmentation processing for ../Dataset/Plum
Done Classfication processing for ../Dataset/Peach saved:  105
Done segmentation processing for ../Dataset/Peach
Done Classfication processing for ../Dataset/Cherry saved:  97
Done segmentation processing for ../Dataset/Cherry
Done Classfication processing for ../Dataset/Potato saved:  104
Done segmentati

Previous system divided each fruit in train/valid/test datasets, now it classification divides on all dataset